# Torontos Neighborhoods

In [1]:
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [2]:
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt

### We import the website

In [3]:
# Library for extracting info from websites, pandas and numpy
import requests
import pandas as pd
import numpy as np
wikilink= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Toronto = requests.get(wikilink)
type(Toronto)
page=Toronto.text
# print(page)

### We locate the beggining and end of the table we want to obtain

In [4]:
# We locate the beggining and end of the table and extract it for further analysis
beg=page.find('<table class="wikitable sortable">')
end=page.find('</tbody></table>')
tabletext=page[beg:end]

#print(tabletext)

### We define this function that will extract the relevant information and return it

In [5]:
# The code esentially differntiates if the row contained a link
def stringsearch(txtrow):

    if textrow.find('">')!=-1:
                #If there is a link to the Borough/Neighborhood
                beg=txtrow.find('">')
                end=txtrow.find('</a>')
                return txtrow[beg+2:end]
    else:
                #No link to the Borugh/Neighborhood
                c=len('<td>')
                if (txtrow[-1]=='\n'):
                    l=len(txtrow)
                    return txtrow[c:l-1]
                else:
                    return txtrow[c:]


### Now we run the text and extract all the information for the dataframe

In [6]:
torontotable=pd.DataFrame(columns=('Postalcode','Borough','Neighborhood'))
X=tabletext.find('</tr>')
searchtext=tabletext[X+4:]
while searchtext.find('<tr>')!=-1:
    A=searchtext.find('<tr>')
    Z=searchtext.find('</tr>')
    #print(A, Z)
    textrow=searchtext[A:Z]
    #print('Row : ', textrow)
    #We find beggining and end of relevant info for postal code
    B=textrow.find('<td>')
    E=textrow.find('</td>')
    #We extract the postalcode info, save it and advance to the borough
    V1=textrow[B+4:E]
    textrow=textrow[E+3:]
    #Select the relevant part with the borough information, apply our function and advance the textrow
    B=textrow.find('<td>')
    E=textrow.find('</td>')
    V2=stringsearch(textrow[B:E])
    textrow=textrow[E+4:]
    #Same as before with neighborhood
    B=textrow.find('<td>')
    E=textrow.find('</td>')
    V3=stringsearch(textrow[B:E])
    #Add our row to the dataframe
    torontotable=torontotable.append({'Postalcode': V1, 'Borough' :V2,'Neighborhood':V3 },ignore_index=True)
    #Move on to the next element
    searchtext=searchtext[Z+3:]

In [7]:
torontotable.head(10)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Dropping the rows withouth a Borough

In [8]:
torontotable.drop(torontotable.loc[torontotable['Borough']=='Not assigned'].index, inplace=True)
torontotable.head(10)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### Fuse the names of Neighborhood with same Postalcode

In [9]:
# We use the variable LPC to keep track of the last postal code that was cheched,
# and if it the same we join the neighborhoods

LPC='000'
torontotable2=pd.DataFrame(columns=('Postalcode','Borough','Neighborhood'))
for row in torontotable.iterrows(): 
    if row[1].Postalcode != LPC:
        tdf=torontotable.loc[torontotable['Postalcode']==row[1].Postalcode]['Neighborhood']
        #print('Postalcode :', row[1].Postalcode, '  //  ', len(tdf), '  //  ', tdf.iloc[0])
        if len(tdf)>1:
            for i in range(1,len(tdf)):
                if row[1].Neighborhood=='Not assigned':
                    row[1].Neighborhood=tdf.iloc[i]
                elif tdf.iloc[i]=='Not assigned':
                    pass
                else:    
                    row[1].Neighborhood=row[1].Neighborhood+', '+tdf.iloc[i]
        torontotable2=torontotable2.append(row[1])
    LPC=row[1].Postalcode
    

In [10]:
torontotable2.head(10)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"
6,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge, Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens, Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson, Garden District"


### Give the neihborhoods without name, the same name of its borough

In [11]:
# look up on editing rows while iterating
for row in torontotable2.iterrows() :
    if row[1].Neighborhood=='Not assigned':
        row[1].Neighborhood=row[1].Borough
       


### Ordering the table by Postalcode, just to be tidy and reseting the index

In [12]:

torontotable2= torontotable2.sort_values(['Postalcode','Borough','Neighborhood'], ascending=[1,0,0])
torontotable2.reset_index(inplace=True)
torontotable2= torontotable2[['Postalcode','Borough','Neighborhood']]
print(torontotable2.shape)
torontotable2.head(10)

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## We Load the CSV file with the longitude and latitude of Borughs in Toronto

In [13]:
url='http://cocl.us/Geospatial_data'
torontogeo=pd.read_csv(url)
torontogeo.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


#### I dont like the space in the column name so I take it away

In [14]:
torontogeo.columns=['Postalcode', 'Latitude', 'Longitude']
torontogeo.head(5)

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Now we merge both tables to obtain the final table

In [15]:
fulltoronto=torontotable2.merge(torontogeo, left_on='Postalcode', right_on='Postalcode')
fulltoronto.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### We import the traffic data

https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/transportation/#7c8e7c62-7630-8b0f-43ed-a2dfe24aadc9

In [16]:
trafficurl='https://www.toronto.ca/ext/open_data/catalog/data_set_files/8hrVeh&PedVolume_6-Mar-2018.xlsx'
trafficdataframe=pd.read_excel(trafficurl)
trafficdataframe.head()

,TCS #,Main,Midblock Route,Side 1 Route,Side 2 Route,Activation Date,Latitude,Longitude,Count Date,8 Peak Hr Vehicle Volume,8 Peak Hr Pedestrian Volume
0,2,JARVIS ST,NaN,FRONT ST E,NaN,11/15/1948,43.649418,-79.371446,2017-06-21,15662,13535
1,3,KING ST E,NaN,JARVIS ST,NaN,08/23/1950,43.650461,-79.371924,2016-09-17,12960,7333
2,4,JARVIS ST,NaN,ADELAIDE ST E,NaN,09/12/1958,43.651534,-79.372360,2016-11-08,17770,7083
3,5,JARVIS ST,NaN,RICHMOND ST E,NaN,04/21/1962,43.652718,-79.372824,2015-12-08,19678,4369
4,6,JARVIS ST,NaN,QUEEN ST E,NaN,08/24/1928,43.653704,-79.373238,2016-09-17,14487,3368


In [17]:
trafficdataframe['ClosestPC']='Nan'
trafficdataframe.shape

(2280, 12)

### We assaign to each location in the traffic data frame the nearest postal area to it

In [18]:
trafficdf=trafficdataframe.copy()
for  row1 in trafficdf.iterrows():
    #We initialize the starting value to a high enough value that all dist will be below it
    shortdist=100
    for  row2 in fulltoronto.iterrows():
        dist=(row1[1].Latitude-row2[1].Latitude)**2+(row1[1].Longitude-row2[1].Longitude)**2
        if dist<shortdist:
            shortdist=dist
            closestpc=row2[1].Postalcode
    #print(row1[0],closestpc)
    #trafficdf.ClosestPC[row1[0]]=closestpc
    #trafficdf['ClosestPC'][row1[0]]=closestpc
    trafficdf.at[row1[0],'ClosestPC']=closestpc



In [19]:
trafficdf.head(10)

,TCS #,Main,Midblock Route,Side 1 Route,Side 2 Route,Activation Date,Latitude,Longitude,Count Date,8 Peak Hr Vehicle Volume,8 Peak Hr Pedestrian Volume,ClosestPC
0,2,JARVIS ST,NaN,FRONT ST E,NaN,11/15/1948,43.649418,-79.371446,2017-06-21,15662,13535,M5C
1,3,KING ST E,NaN,JARVIS ST,NaN,08/23/1950,43.650461,-79.371924,2016-09-17,12960,7333,M5C
2,4,JARVIS ST,NaN,ADELAIDE ST E,NaN,09/12/1958,43.651534,-79.372360,2016-11-08,17770,7083,M5C
3,5,JARVIS ST,NaN,RICHMOND ST E,NaN,04/21/1962,43.652718,-79.372824,2015-12-08,19678,4369,M5C
4,6,JARVIS ST,NaN,QUEEN ST E,NaN,08/24/1928,43.653704,-79.373238,2016-09-17,14487,3368,M5C
5,7,JARVIS ST,NaN,SHUTER ST,NaN,11/18/1948,43.655357,-79.373862,2016-11-08,15846,3747,M5C
6,8,JARVIS ST,NaN,DUNDAS ST E,NaN,06/21/1928,43.657052,-79.374531,2017-06-27,17835,5858,M5B
7,9,JARVIS ST,NaN,GERRARD ST E,NaN,07/14/1941,43.660432,-79.375854,2016-11-01,18196,6493,M5B
8,10,JARVIS ST,NaN,CARLTON ST,NaN,06/28/1928,43.662420,-79.376708,2017-01-21,14222,6165,M5B
9,11,JARVIS ST,NaN,WELLESLEY ST E,NaN,09/28/1948,43.666289,-79.378325,2016-11-01,18163,5213,M4Y


In [20]:
trafficdf.tail()

,TCS #,Main,Midblock Route,Side 1 Route,Side 2 Route,Activation Date,Latitude,Longitude,Count Date,8 Peak Hr Vehicle Volume,8 Peak Hr Pedestrian Volume,ClosestPC
2275,3005,WINDERMERE AVE,NaN,MORNINGSIDE AVE,NaN,01/01/1967,43.644885,-79.478592,2009-12-10,6488,634,M6S
2276,3006,GATEWAY BLVD,NaN,GRENOBLE DR,NaN,05/28/1976,43.711967,-79.330476,2017-05-02,7447,1316,M3C
2277,3009,ALLIANCE AVE,NaN,ROCKCLIFFE BLVD,NaN,02/13/1969,43.676947,-79.489360,2018-02-15,8780,498,M6N
2278,3011,OLD FINCH AVE,250m EAST OF,SEWELLS RD,NaN,04/11/1974,43.825259,-79.196812,2008-11-27,1081,0,M1X
2279,3014,ZOO RD,NaN,MEADOWVALE RD N ZOO W RAMP,ZOO MEADOWVALE RD N RAMP,02/08/1974,43.818187,-79.173580,2009-07-26,1982,16,M1B


In [21]:
trafficdf.ClosestPC.get_values()

array(['M5C', 'M5C', 'M5C', ..., 'M6N', 'M1X', 'M1B'], dtype=object)

### And we assign to each postal code area the mean of those locations closest to said postal code area

In [22]:
meantraffic=trafficdf.groupby(['ClosestPC']).mean()
print(meantraffic.columns)
meantraffic.head()

Index(['TCS #', 'Latitude', 'Longitude', '8 Peak Hr Vehicle Volume',
       '8 Peak Hr Pedestrian Volume'],
      dtype='object')


,TCS #,Latitude,Longitude,8 Peak Hr Vehicle Volume,8 Peak Hr Pedestrian Volume
ClosestPC,,,,,
M1B,1686.315789,43.803058,-79.202146,11453.552632,725.052632
M1C,1629.120000,43.784289,-78.923751,9436.520000,457.640000
M1E,1160.285714,43.765703,-79.189726,15733.714286,781.666667
M1G,1288.392857,43.770180,-79.217162,14060.785714,706.035714
M1H,1333.400000,43.779115,-79.242165,18679.600000,584.057143


In [23]:
meantraffic=meantraffic.reset_index()
print(meantraffic.shape)
meantraffic.head()

(103, 6)


,ClosestPC,TCS #,Latitude,Longitude,8 Peak Hr Vehicle Volume,8 Peak Hr Pedestrian Volume
0,M1B,1686.315789,43.803058,-79.202146,11453.552632,725.052632
1,M1C,1629.120000,43.784289,-78.923751,9436.520000,457.640000
2,M1E,1160.285714,43.765703,-79.189726,15733.714286,781.666667
3,M1G,1288.392857,43.770180,-79.217162,14060.785714,706.035714
4,M1H,1333.400000,43.779115,-79.242165,18679.600000,584.057143


In [24]:
meantraffic.columns=['ClosestPC', 'TCS #', 'Lat', 'Lon',
       '8 Peak Hr Veh Vol', '8 Peak Hr Ped Vol']
meantraffic.head()

,ClosestPC,TCS #,Lat,Lon,8 Peak Hr Veh Vol,8 Peak Hr Ped Vol
0,M1B,1686.315789,43.803058,-79.202146,11453.552632,725.052632
1,M1C,1629.120000,43.784289,-78.923751,9436.520000,457.640000
2,M1E,1160.285714,43.765703,-79.189726,15733.714286,781.666667
3,M1G,1288.392857,43.770180,-79.217162,14060.785714,706.035714
4,M1H,1333.400000,43.779115,-79.242165,18679.600000,584.057143


### We merge the mean traffic in each postal code area with the information of the Borough and Neighborhoods

In [25]:
torontotraffic=fulltoronto.merge(meantraffic, left_on='Postalcode', right_on='ClosestPC')
torontotraffic.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude,ClosestPC,TCS #,Lat,Lon,8 Peak Hr Veh Vol,8 Peak Hr Ped Vol
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,M1B,1686.315789,43.803058,-79.202146,11453.552632,725.052632
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,M1C,1629.120000,43.784289,-78.923751,9436.520000,457.640000
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,M1E,1160.285714,43.765703,-79.189726,15733.714286,781.666667
3,M1G,Scarborough,Woburn,43.770992,-79.216917,M1G,1288.392857,43.770180,-79.217162,14060.785714,706.035714
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,M1H,1333.400000,43.779115,-79.242165,18679.600000,584.057143
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,M1J,1066.272727,43.745631,-79.235932,16726.090909,915.272727
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,M1K,1222.322581,43.728425,-79.263789,14189.129032,741.709677
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,M1L,1339.000000,43.713832,-79.287035,14976.093750,775.156250
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,M1M,952.142857,43.718548,-79.242370,17438.142857,639.000000
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,M1N,789.789474,43.696907,-79.269347,12161.736842,407.736842


In [26]:
torontotraffic.columns

Index(['Postalcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude',
       'ClosestPC', 'TCS #', 'Lat', 'Lon', '8 Peak Hr Veh Vol',
       '8 Peak Hr Ped Vol'],
      dtype='object')

### We select those columns we are interested in

In [27]:
torontotraff=torontotraffic[['Postalcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude',
                             '8 Peak Hr Veh Vol','8 Peak Hr Ped Vol']].copy()
print(torontotraff.shape)
torontotraff.head()

(103, 7)


,Postalcode,Borough,Neighborhood,Latitude,Longitude,8 Peak Hr Veh Vol,8 Peak Hr Ped Vol
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,11453.552632,725.052632
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,9436.520000,457.640000
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,15733.714286,781.666667
3,M1G,Scarborough,Woburn,43.770992,-79.216917,14060.785714,706.035714
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,18679.600000,584.057143


### Paramaters to call the foursquare API

In [28]:
# @Hidden_cell
CLIENT_ID = '3TSE5R42NL5B4JHO3HNR2ZDTVQY2JWND5GT4YSIE2OCLAUBC' # your Foursquare ID
CLIENT_SECRET = '5AOGAUDRIHEFVGJ5MRKB0XWT05FBLDEAWHDH2U1JNPZ42BWO' # your Foursquare Secret
VERSION = '20180604'


In [29]:
search_query = 'Restaurant'
radius = 150
LIMIT = 30

latitude=fulltoronto.Latitude[77]
longitude=fulltoronto.Longitude[77]

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=3TSE5R42NL5B4JHO3HNR2ZDTVQY2JWND5GT4YSIE2OCLAUBC&client_secret=5AOGAUDRIHEFVGJ5MRKB0XWT05FBLDEAWHDH2U1JNPZ42BWO&ll=43.647926700000006,-79.4197497&v=20180604&query=Restaurant&radius=150&limit=30'

### We made a test call to Borough number 78 on the list

In [30]:
results = requests.get(url).json()
# results

In [31]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
print('Total results: ',dataframe.shape[0])

Total results:  4


### The function that cleans up the data obtained from the json file

In [32]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.shape[0]

4

### We take the subtable that contain the word 'Toronto' in the Borough column

In [33]:
onlytoronto=torontotraff[torontotraff['Borough'].str.contains('Toronto')].copy()
onlytoronto.reset_index(inplace=True)
onlytoronto=onlytoronto[['Postalcode','Borough','Neighborhood','Latitude','Longitude',
                         '8 Peak Hr Veh Vol','8 Peak Hr Ped Vol']].copy()
onlytoronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,8 Peak Hr Veh Vol,8 Peak Hr Ped Vol
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,12703.153846,2070.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,12302.062500,3862.875000
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,11970.444444,1488.777778
3,M4M,East Toronto,Studio District,43.659526,-79.340923,11335.000000,1715.240000
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,19865.333333,725.333333


### We call the Foursquare API searching for restaurants en each Borough and keep a track of all the restaurants found

In [34]:
Count=[]
for index,row in onlytoronto.iterrows():
    latitude=row.Latitude
    longitude=row.Longitude
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    if index==0:
        dataframe = json_normalize(venues)
        dataframe['Postalcode']=row.Postalcode
        Count=Count+[dataframe.shape[0]]
    else:
        tempdataframe = json_normalize(venues)
        tempdataframe['Postalcode']=row.Postalcode
 
        dataframe=pd.concat([dataframe,tempdataframe],sort=True)
        Count=Count+[tempdataframe.shape[0]]
    



In [35]:
print(Count[0:10])
print(dataframe.shape)
dataframe.head()

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
(57, 20)


,Postalcode,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,M4P,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",False,4adb2fd3f964a520c42421e3,955 Mount Pleasant,CA,Toronto,Canada,Mount Pleasant & Erskine,110.0,[955 Mount Pleasant (Mount Pleasant & Erskine)...,"[{'label': 'display', 'lat': 43.71264120397444...",43.712641,-79.391557,NaN,M4P 2L7,ON,Homeway Restaurant & Brunch,v-1550500520,NaN
0,M4X,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4b7b268cf964a52061542fe3,580 Parliament St,CA,Toronto,Canada,at Wellesley St,147.0,"[580 Parliament St (at Wellesley St), Toronto ...","[{'label': 'display', 'lat': 43.667662, 'lng':...",43.667662,-79.369464,NaN,M4X 1P8,ON,Tender Trap Restaurant,v-1550500521,NaN
1,M4X,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4c4b1e1ac668e21ea05da0f9,587 parliament street,CA,Toronto,Canada,Parliament and wellesley,148.0,[587 parliament street (Parliament and wellesl...,"[{'label': 'display', 'lat': 43.66741592867017...",43.667416,-79.369350,NaN,NaN,ON,Rainbow Restaurant,v-1550500521,NaN
0,M4Y,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",False,4b2d27b4f964a520a6cf24e3,574 Yonge St.,CA,Toronto,Canada,Wellesley,164.0,"[574 Yonge St. (Wellesley), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.66499059853204...",43.664991,-79.384814,NaN,NaN,ON,Coach House Restaurant,v-1550500521,NaN
0,M5A,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",False,4ae5b91ff964a520a6a121e3,457 King St. E,CA,Toronto,Canada,Gilead Place,54.0,"[457 King St. E (Gilead Place), Toronto ON M5A...","[{'label': 'display', 'lat': 43.65394694263529...",43.653947,-79.361149,NaN,M5A 1L6,ON,Morning Glory Cafe,v-1550500522,39686393


### We clean up the data with the get_category_type function

In [36]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['Postalcode', 'name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]



# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,Postalcode,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,M4P,Homeway Restaurant & Brunch,Breakfast Spot,955 Mount Pleasant,CA,Toronto,Canada,Mount Pleasant & Erskine,110.0,[955 Mount Pleasant (Mount Pleasant & Erskine)...,"[{'label': 'display', 'lat': 43.71264120397444...",43.712641,-79.391557,NaN,M4P 2L7,ON,4adb2fd3f964a520c42421e3
0,M4X,Tender Trap Restaurant,Chinese Restaurant,580 Parliament St,CA,Toronto,Canada,at Wellesley St,147.0,"[580 Parliament St (at Wellesley St), Toronto ...","[{'label': 'display', 'lat': 43.667662, 'lng':...",43.667662,-79.369464,NaN,M4X 1P8,ON,4b7b268cf964a52061542fe3
1,M4X,Rainbow Restaurant,Indian Restaurant,587 parliament street,CA,Toronto,Canada,Parliament and wellesley,148.0,[587 parliament street (Parliament and wellesl...,"[{'label': 'display', 'lat': 43.66741592867017...",43.667416,-79.369350,NaN,NaN,ON,4c4b1e1ac668e21ea05da0f9
0,M4Y,Coach House Restaurant,Diner,574 Yonge St.,CA,Toronto,Canada,Wellesley,164.0,"[574 Yonge St. (Wellesley), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.66499059853204...",43.664991,-79.384814,NaN,NaN,ON,4b2d27b4f964a520a6cf24e3
0,M5A,Morning Glory Cafe,Breakfast Spot,457 King St. E,CA,Toronto,Canada,Gilead Place,54.0,"[457 King St. E (Gilead Place), Toronto ON M5A...","[{'label': 'display', 'lat': 43.65394694263529...",43.653947,-79.361149,NaN,M5A 1L6,ON,4ae5b91ff964a520a6a121e3


In [37]:
restlist=dataframe_filtered.copy()
restlist.columns

Index(['Postalcode', 'name', 'categories', 'address', 'cc', 'city', 'country',
       'crossStreet', 'distance', 'formattedAddress', 'labeledLatLngs', 'lat',
       'lng', 'neighborhood', 'postalCode', 'state', 'id'],
      dtype='object')

### We reset the index and choose the columns that we are interested in

In [38]:

restlist.reset_index(inplace=True)
restlist=restlist[['Postalcode', 'name', 'categories', 'address', 
       'crossStreet', 'distance', 'formattedAddress', 'lat',
       'lng', 'state', 'id']]
print(restlist.shape)
restlist.head(10)

(57, 11)


,Postalcode,name,categories,address,crossStreet,distance,formattedAddress,lat,lng,state,id
0,M4P,Homeway Restaurant & Brunch,Breakfast Spot,955 Mount Pleasant,Mount Pleasant & Erskine,110.0,[955 Mount Pleasant (Mount Pleasant & Erskine)...,43.712641,-79.391557,ON,4adb2fd3f964a520c42421e3
1,M4X,Tender Trap Restaurant,Chinese Restaurant,580 Parliament St,at Wellesley St,147.0,"[580 Parliament St (at Wellesley St), Toronto ...",43.667662,-79.369464,ON,4b7b268cf964a52061542fe3
2,M4X,Rainbow Restaurant,Indian Restaurant,587 parliament street,Parliament and wellesley,148.0,[587 parliament street (Parliament and wellesl...,43.667416,-79.369350,ON,4c4b1e1ac668e21ea05da0f9
3,M4Y,Coach House Restaurant,Diner,574 Yonge St.,Wellesley,164.0,"[574 Yonge St. (Wellesley), Toronto ON, Canada]",43.664991,-79.384814,ON,4b2d27b4f964a520a6cf24e3
4,M5A,Morning Glory Cafe,Breakfast Spot,457 King St. E,Gilead Place,54.0,"[457 King St. E (Gilead Place), Toronto ON M5A...",43.653947,-79.361149,ON,4ae5b91ff964a520a6a121e3
5,M5B,The Senator Restaurant,Diner,249 Victoria Street,btwn Dundas St E and Shuter St,170.0,[249 Victoria Street (btwn Dundas St E and Shu...,43.655641,-79.379199,ON,4ad7929cf964a520500c21e3
6,M5B,Subway,Sandwich Place,10 Dundas St. E,at Yonge St.,188.0,"[10 Dundas St. E (at Yonge St.), Toronto ON M5...",43.656260,-79.380920,ON,4c6abb8c2c29d13ad2720e41
7,M5C,Golden Thai Restaurant,Thai Restaurant,105 Church St,at Richmond St E,114.0,"[105 Church St (at Richmond St E), Toronto ON ...",43.652525,-79.375369,ON,4ad4c061f964a520a7f720e3
8,M5C,L'Apero Restaurant,Restaurant,81 Church Street,NaN,88.0,"[81 Church Street, Toronto ON, Canada]",43.651820,-79.374410,ON,4dff2649d22d48be8127cd1c
9,M5C,B.L.D Restaurant,New American Restaurant,36 Toronto St,Adelaide,122.0,"[36 Toronto St (Adelaide), Toronto ON, Canada]",43.650832,-79.376639,ON,4c38f3fc0a71c9b6f6ad41c9


### Now we eliminate the duplicate restaurants and keep the Postalcode area with the shorter distance 

In [39]:
restlist=restlist.sort_values('distance',ascending=False)
restlist=restlist.drop_duplicates(subset='name', keep='last')
restlist=restlist.reset_index(drop=True)
print(restlist.shape)
restlist.head()

(52, 11)


,Postalcode,name,categories,address,crossStreet,distance,formattedAddress,lat,lng,state,id
0,M5B,Subway,Sandwich Place,10 Dundas St. E,at Yonge St.,188.0,"[10 Dundas St. E (at Yonge St.), Toronto ON M5...",43.656260,-79.380920,ON,4c6abb8c2c29d13ad2720e41
1,M5K,Epic,Wine Bar,100 Front Street West,NaN,187.0,"[100 Front Street West, Toronto ON M5J 1E3, Ca...",43.645515,-79.381239,ON,4ad4c05df964a5203af620e3
2,M5K,McDonald's,Fast Food Restaurant,61 Front St. W.,in Union Station,182.0,"[61 Front St. W. (in Union Station), Toronto O...",43.646146,-79.379822,ON,4b8d36f8f964a520c7ed32e3
3,M5L,Ninki Sushi,Japanese Restaurant,40 King St W,at Bay St,181.0,"[40 King St W (at Bay St), Toronto ON M5H 1A1,...",43.649812,-79.379518,ON,4beb2c0ba9900f475b7a1740
4,M5L,Over Easy,Breakfast Spot,56 Yonge St,at Melinda St,179.0,"[56 Yonge St (at Melinda St), Toronto ON M5E 1...",43.648077,-79.377599,ON,4b06e099f964a52002f222e3


### Now we add the number of restaurants found in each Borough to our database

In [40]:
pccount=restlist.groupby('Postalcode').count()
pccount=pccount.reset_index(drop=False)
pccount=pccount[['Postalcode','name']].copy()
pccount.columns=['Postalcode','NumberRestaurants']
pccount

,Postalcode,NumberRestaurants
0,M4P,1
1,M4X,2
2,M4Y,1
3,M5A,1
4,M5B,2
5,M5C,6
6,M5H,3
7,M5J,1
8,M5K,4
9,M5L,6


In [41]:
onlytoronto=onlytoronto.merge(pccount, left_on='Postalcode', right_on='Postalcode', how='left')
onlytoronto['NumberRestaurants'].fillna(0, inplace=True)
onlytoronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,8 Peak Hr Veh Vol,8 Peak Hr Ped Vol,NumberRestaurants
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,12703.153846,2070.000000,0.0
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,12302.062500,3862.875000,0.0
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,11970.444444,1488.777778,0.0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,11335.000000,1715.240000,0.0
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,19865.333333,725.333333,0.0


### We import the Kmeans algorithm

In [42]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs


## We are going to make 3 clusters based on the amount of restaurants in each Postalcode area

In [43]:
onlytorontocopy=onlytoronto.copy()

In [44]:
X=onlytorontocopy[['NumberRestaurants']]
X=X.astype(float)
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
# cluster_dataset

In [45]:
num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 2 2 1 1 1 1 0 1 1 1 1 1 2 1 1 1 1
 1]


### We are going to make 4 clusters based on the average pedestrian and vehicule traffic in each Postalcode area

In [46]:
X2=onlytorontocopy[['8 Peak Hr Veh Vol','8 Peak Hr Ped Vol']]
X2=X2.astype(float)
X2 = np.nan_to_num(X2)
cluster_dataset2 = StandardScaler().fit_transform(X2)
# cluster_dataset

In [47]:
num_clusters = 4

k_means2 = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means2.fit(cluster_dataset2)
labels2 = k_means2.labels_

print(labels2)

[0 0 0 0 2 2 2 2 2 2 0 0 3 0 3 3 0 1 3 0 1 1 2 0 2 3 3 2 3 3 0 0 0 0 0 2 0
 0]


### We assign the cluster label to the dataframe

In [48]:
restdentoronto=onlytorontocopy.copy()
restdentoronto['labels']=labels
restdentoronto['labels2']=labels2
restdentoronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,8 Peak Hr Veh Vol,8 Peak Hr Ped Vol,NumberRestaurants,labels,labels2
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,12703.153846,2070.000000,0.0,1,0
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,12302.062500,3862.875000,0.0,1,0
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,11970.444444,1488.777778,0.0,1,0
3,M4M,East Toronto,Studio District,43.659526,-79.340923,11335.000000,1715.240000,0.0,1,0
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,19865.333333,725.333333,0.0,1,2


In [49]:
restdentoronto.columns=['Postalcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude','VehVol', 'PedVol', 'NumberRestaurants', 'labels','labels2']
restdentoronto.columns   

Index(['Postalcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude',
       'VehVol', 'PedVol', 'NumberRestaurants', 'labels', 'labels2'],
      dtype='object')

In [50]:
restdentoronto.tail()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,VehVol,PedVol,NumberRestaurants,labels,labels2
33,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,10931.307692,1839.307692,0.0,1,0
34,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,12047.111111,1367.740741,0.0,1,0
35,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,16636.629630,1297.074074,0.0,1,2
36,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,11637.571429,2491.571429,0.0,1,0
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,11742.166667,1023.833333,0.0,1,0


### Now we subselect those Postalcode areas with highest amount of foot traffic

In [57]:
# Each time we run the Kmeans algorithm we have to see what value the labels have
hottoronto=restdentoronto.copy()
hottoronto=hottoronto.loc[hottoronto['labels2'].isin([1,3])]
hottoronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,VehVol,PedVol,NumberRestaurants,labels,labels2
12,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,12406.000000,9529.172414,1.0,1,3
14,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,11362.473684,10737.789474,2.0,1,3
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,11973.000000,9141.923077,6.0,2,3
17,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,17242.090909,17227.818182,0.0,1,1
18,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,11969.916667,11662.083333,3.0,2,3


In [58]:
hotcodes=hottoronto['Postalcode']
hotcodes.head()

12    M4Y
14    M5B
15    M5C
17    M5G
18    M5H
Name: Postalcode, dtype: object

### And select the restaurants in these areas

In [59]:
hotrest=restlist[restlist.Postalcode.isin(hotcodes)]
hotrest=hotrest.copy()
hotrest.columns

Index(['Postalcode', 'name', 'categories', 'address', 'crossStreet',
       'distance', 'formattedAddress', 'lat', 'lng', 'state', 'id'],
      dtype='object')

In [60]:
hotrest.reset_index(inplace=True)
hotrest=hotrest[['Postalcode', 'name', 'categories', 'address', 'crossStreet',
       'distance', 'lat', 'lng',  'id']]
hotrest.head(10)

,Postalcode,name,categories,address,crossStreet,distance,lat,lng,id
0,M5B,Subway,Sandwich Place,10 Dundas St. E,at Yonge St.,188.0,43.656260,-79.380920,4c6abb8c2c29d13ad2720e41
1,M5K,Epic,Wine Bar,100 Front Street West,NaN,187.0,43.645515,-79.381239,4ad4c05df964a5203af620e3
2,M5K,McDonald's,Fast Food Restaurant,61 Front St. W.,in Union Station,182.0,43.646146,-79.379822,4b8d36f8f964a520c7ed32e3
3,M5L,Ninki Sushi,Japanese Restaurant,40 King St W,at Bay St,181.0,43.649812,-79.379518,4beb2c0ba9900f475b7a1740
4,M5L,Over Easy,Breakfast Spot,56 Yonge St,at Melinda St,179.0,43.648077,-79.377599,4b06e099f964a52002f222e3
5,M5B,The Senator Restaurant,Diner,249 Victoria Street,btwn Dundas St E and Shuter St,170.0,43.655641,-79.379199,4ad7929cf964a520500c21e3
6,M5L,Vineyard Jamaican Restaurant,Caribbean Restaurant,NaN,NaN,169.0,43.649550,-79.380780,4ba2ab4af964a520650e38e3
7,M5T,Rol San Restaurant 龍笙棧,Dim Sum Restaurant,323 Spadina Ave.,at D'Arcy St.,167.0,43.654318,-79.398650,4ad4c060f964a5207ff720e3
8,M5T,Pho Hung,Vietnamese Restaurant,350 Spadina Ave.,at St Andrews St.,165.0,43.654434,-79.398883,4ad4c060f964a52056f720e3
9,M4Y,Coach House Restaurant,Diner,574 Yonge St.,Wellesley,164.0,43.664991,-79.384814,4b2d27b4f964a520a6cf24e3


In [61]:
print('Number of restaurants in the hottest part of town:',hotrest.shape[0])

Number of restaurants in the hottest part of town: 40


In [62]:

# Test call to see if daily foursquare quota is exceeded
venue_id = '4ae38f3ff964a520ab9621e3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()
print(result)

try:
    x=result['response']['venue']['rating']
    print(x)
except:
    x='This venue has not been rated yet.'
    print(x)


{'meta': {'code': 200, 'requestId': '5c6ac30e4c1f6762dc6afd24'}, 'response': {'venue': {'id': '4ae38f3ff964a520ab9621e3', 'name': 'Honey Bee Restaurant', 'contact': {'phone': '4166985567', 'formattedPhone': '(416) 698-5567'}, 'location': {'address': '2028 Queen St E.', 'crossStreet': 'Lee Ave', 'lat': 43.670409, 'lng': -79.298406, 'labeledLatLngs': [{'label': 'display', 'lat': 43.670409, 'lng': -79.298406}], 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['2028 Queen St E. (Lee Ave)', 'Toronto ON', 'Canada']}, 'canonicalUrl': 'https://foursquare.com/v/honey-bee-restaurant/4ae38f3ff964a520ab9621e3', 'categories': [{'id': '4bf58dd8d48988d145941735', 'name': 'Chinese Restaurant', 'pluralName': 'Chinese Restaurants', 'shortName': 'Chinese', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}], 'verified': False, 'stats': {'tipCount': 3}, 'price': {'tier': 4, 'message': 'Very Expensive', 'curren

### We collect each restaurants total number of tips and average ratings from foursquare

In [63]:
print(hotrest.shape)
hotrest.columns

(40, 9)


Index(['Postalcode', 'name', 'categories', 'address', 'crossStreet',
       'distance', 'lat', 'lng', 'id'],
      dtype='object')

In [64]:
## Changed hotrest for restlist when assigning values
for index, row in hotrest.iterrows():
    venue_id = '{}'.format(row.id) 
    name='{}'.format(hotrest.name[index]) 
    print(name,' id number: ',venue_id)
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()

    hotrest.loc[index, 'numbertips']=result['response']['venue']['tips']['count']
    try:
        hotrest.loc[index, 'rating']=result['response']['venue']['rating'] 
        print(name, ' has a rating of ',result['response']['venue']['rating'] )
    except:
        print(name, ' has no rating  ')
    print('Number of tips in ', name,': ', result['response']['venue']['tips']['count'])


Subway  id number:  4c6abb8c2c29d13ad2720e41
Subway  has a rating of  5.8
Number of tips in  Subway :  3
Epic  id number:  4ad4c05df964a5203af620e3
Epic  has a rating of  6.9
Number of tips in  Epic :  28
McDonald's  id number:  4b8d36f8f964a520c7ed32e3
McDonald's  has a rating of  4.7
Number of tips in  McDonald's :  8
Ninki Sushi  id number:  4beb2c0ba9900f475b7a1740
Ninki Sushi  has a rating of  5.7
Number of tips in  Ninki Sushi :  15
Over Easy  id number:  4b06e099f964a52002f222e3
Over Easy  has a rating of  7.7
Number of tips in  Over Easy :  32
The Senator Restaurant  id number:  4ad7929cf964a520500c21e3
The Senator Restaurant  has a rating of  8.2
Number of tips in  The Senator Restaurant :  94
Vineyard Jamaican Restaurant  id number:  4ba2ab4af964a520650e38e3
Vineyard Jamaican Restaurant  has no rating  
Number of tips in  Vineyard Jamaican Restaurant :  0
Rol San Restaurant 龍笙棧  id number:  4ad4c060f964a5207ff720e3
Rol San Restaurant 龍笙棧  has a rating of  7.5
Number of tips i

In [65]:
print(hotrest.shape)
print(hotrest.columns)
hotrest.head()

(40, 11)
Index(['Postalcode', 'name', 'categories', 'address', 'crossStreet',
       'distance', 'lat', 'lng', 'id', 'numbertips', 'rating'],
      dtype='object')


,Postalcode,name,categories,address,crossStreet,distance,lat,lng,id,numbertips,rating
0,M5B,Subway,Sandwich Place,10 Dundas St. E,at Yonge St.,188.0,43.656260,-79.380920,4c6abb8c2c29d13ad2720e41,3.0,5.8
1,M5K,Epic,Wine Bar,100 Front Street West,NaN,187.0,43.645515,-79.381239,4ad4c05df964a5203af620e3,28.0,6.9
2,M5K,McDonald's,Fast Food Restaurant,61 Front St. W.,in Union Station,182.0,43.646146,-79.379822,4b8d36f8f964a520c7ed32e3,8.0,4.7
3,M5L,Ninki Sushi,Japanese Restaurant,40 King St W,at Bay St,181.0,43.649812,-79.379518,4beb2c0ba9900f475b7a1740,15.0,5.7
4,M5L,Over Easy,Breakfast Spot,56 Yonge St,at Melinda St,179.0,43.648077,-79.377599,4b06e099f964a52002f222e3,32.0,7.7


In [75]:
cleanrest=hotrest[['Postalcode', 'name', 'categories', 'address', 'crossStreet',
       'distance', 'lat', 'lng', 'id', 'numbertips', 'rating']].copy()


### We assign a rating of 5 to those reataurants that haven't been rated

In [76]:
cleanrest['rating'].fillna(5, inplace=True)

### We define the profitrate as the restaurants rating times the number of tips on foursquare

In [77]:
cleanrest['profitrate']=cleanrest['numbertips']*cleanrest['rating']
cleanrest['profitrate'].fillna(0, inplace=True)
print(cleanrest.shape)
cleanrest.head()

(40, 12)


,Postalcode,name,categories,address,crossStreet,distance,lat,lng,id,numbertips,rating,profitrate
0,M5B,Subway,Sandwich Place,10 Dundas St. E,at Yonge St.,188.0,43.656260,-79.380920,4c6abb8c2c29d13ad2720e41,3.0,5.8,17.4
1,M5K,Epic,Wine Bar,100 Front Street West,NaN,187.0,43.645515,-79.381239,4ad4c05df964a5203af620e3,28.0,6.9,193.2
2,M5K,McDonald's,Fast Food Restaurant,61 Front St. W.,in Union Station,182.0,43.646146,-79.379822,4b8d36f8f964a520c7ed32e3,8.0,4.7,37.6
3,M5L,Ninki Sushi,Japanese Restaurant,40 King St W,at Bay St,181.0,43.649812,-79.379518,4beb2c0ba9900f475b7a1740,15.0,5.7,85.5
4,M5L,Over Easy,Breakfast Spot,56 Yonge St,at Melinda St,179.0,43.648077,-79.377599,4b06e099f964a52002f222e3,32.0,7.7,246.4


In [78]:
cleanrest=cleanrest.join(pd.get_dummies(cleanrest['categories']))
cleanrest.head()

,Postalcode,name,categories,address,crossStreet,distance,lat,lng,id,numbertips,...,Italian Restaurant,Japanese Restaurant,Korean Restaurant,New American Restaurant,Nightclub,Restaurant,Sandwich Place,Thai Restaurant,Vietnamese Restaurant,Wine Bar
0,M5B,Subway,Sandwich Place,10 Dundas St. E,at Yonge St.,188.0,43.656260,-79.380920,4c6abb8c2c29d13ad2720e41,3.0,...,0,0,0,0,0,0,1,0,0,0
1,M5K,Epic,Wine Bar,100 Front Street West,NaN,187.0,43.645515,-79.381239,4ad4c05df964a5203af620e3,28.0,...,0,0,0,0,0,0,0,0,0,1
2,M5K,McDonald's,Fast Food Restaurant,61 Front St. W.,in Union Station,182.0,43.646146,-79.379822,4b8d36f8f964a520c7ed32e3,8.0,...,0,0,0,0,0,0,0,0,0,0
3,M5L,Ninki Sushi,Japanese Restaurant,40 King St W,at Bay St,181.0,43.649812,-79.379518,4beb2c0ba9900f475b7a1740,15.0,...,0,1,0,0,0,0,0,0,0,0
4,M5L,Over Easy,Breakfast Spot,56 Yonge St,at Melinda St,179.0,43.648077,-79.377599,4b06e099f964a52002f222e3,32.0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
print(cleanrest.columns)
print(cleanrest.shape)

Index(['Postalcode', 'name', 'categories', 'address', 'crossStreet',
       'distance', 'lat', 'lng', 'id', 'numbertips', 'rating', 'profitrate',
       'American Restaurant', 'Bar', 'Breakfast Spot', 'Caribbean Restaurant',
       'Chinese Restaurant', 'Dim Sum Restaurant', 'Diner',
       'Dumpling Restaurant', 'Fast Food Restaurant', 'Gastropub',
       'Italian Restaurant', 'Japanese Restaurant', 'Korean Restaurant',
       'New American Restaurant', 'Nightclub', 'Restaurant', 'Sandwich Place',
       'Thai Restaurant', 'Vietnamese Restaurant', 'Wine Bar'],
      dtype='object')
(40, 32)


### We group the different restaurant categories

These groups might change each time we run the code, depending on the foursquare results. We can look at the columns of the cleanrest dataframe and group them in to more generic categories.
Be aware if new groups are created some of the following code will need some adjustement.

In [87]:
cleanrest['AsianRestaurant']=0
cleanrest['Bar/Club']=0
cleanrest['AmericanRestaurant']=0
cleanrest['OtherRestaurant']=0

In [89]:
#these might change depending on the categories of the foursquare search
cleanrest['AsianRestaurant']=cleanrest['Japanese Restaurant']+cleanrest['Chinese Restaurant']+cleanrest['Vietnamese Restaurant']+cleanrest['Dim Sum Restaurant']+cleanrest['Dumpling Restaurant']+cleanrest['Korean Restaurant']+cleanrest['Thai Restaurant']
#+cleanrest['Indian Restaurant']+cleanrest['Asian Restaurant']
cleanrest['Bar/Club']=cleanrest['Nightclub']+cleanrest['Bar']+cleanrest['Gastropub']+cleanrest['Wine Bar']
cleanrest['AmericanRestaurant']=cleanrest['Restaurant']+cleanrest['New American Restaurant']+cleanrest['Sandwich Place']+cleanrest['Fast Food Restaurant']+cleanrest['American Restaurant']
#+cleanrest['Deli / Bodega']
cleanrest['OtherRestaurant']=cleanrest['Italian Restaurant']+cleanrest['Diner']+cleanrest['Caribbean Restaurant']
#+cleanrest['Portuguese Restaurant']

In [90]:
#these might change depending on the categories of the foursquare search
tidyrest=cleanrest[['Postalcode', 'name', 'categories', 'address', 'crossStreet',
       'distance', 'lat', 'lng', 'id', 'numbertips', 'rating', 'profitrate','AmericanRestaurant','Breakfast Spot','OtherRestaurant','AsianRestaurant','Bar/Club']].copy()
tidyrest.head()

,Postalcode,name,categories,address,crossStreet,distance,lat,lng,id,numbertips,rating,profitrate,AmericanRestaurant,Breakfast Spot,OtherRestaurant,AsianRestaurant,Bar/Club
0,M5B,Subway,Sandwich Place,10 Dundas St. E,at Yonge St.,188.0,43.656260,-79.380920,4c6abb8c2c29d13ad2720e41,3.0,5.8,17.4,1,0,0,0,0
1,M5K,Epic,Wine Bar,100 Front Street West,NaN,187.0,43.645515,-79.381239,4ad4c05df964a5203af620e3,28.0,6.9,193.2,0,0,0,0,1
2,M5K,McDonald's,Fast Food Restaurant,61 Front St. W.,in Union Station,182.0,43.646146,-79.379822,4b8d36f8f964a520c7ed32e3,8.0,4.7,37.6,1,0,0,0,0
3,M5L,Ninki Sushi,Japanese Restaurant,40 King St W,at Bay St,181.0,43.649812,-79.379518,4beb2c0ba9900f475b7a1740,15.0,5.7,85.5,0,0,0,1,0
4,M5L,Over Easy,Breakfast Spot,56 Yonge St,at Melinda St,179.0,43.648077,-79.377599,4b06e099f964a52002f222e3,32.0,7.7,246.4,0,1,0,0,0


In [91]:
fullrest=tidyrest.merge(hottoronto,left_on='Postalcode',right_on='Postalcode')
print(fullrest.columns)
fullrest.head()

Index(['Postalcode', 'name', 'categories', 'address', 'crossStreet',
       'distance', 'lat', 'lng', 'id', 'numbertips', 'rating', 'profitrate',
       'AmericanRestaurant', 'Breakfast Spot', 'OtherRestaurant',
       'AsianRestaurant', 'Bar/Club', 'Borough', 'Neighborhood', 'Latitude',
       'Longitude', 'VehVol', 'PedVol', 'NumberRestaurants', 'labels',
       'labels2'],
      dtype='object')


,Postalcode,name,categories,address,crossStreet,distance,lat,lng,id,numbertips,...,Bar/Club,Borough,Neighborhood,Latitude,Longitude,VehVol,PedVol,NumberRestaurants,labels,labels2
0,M5B,Subway,Sandwich Place,10 Dundas St. E,at Yonge St.,188.0,43.656260,-79.380920,4c6abb8c2c29d13ad2720e41,3.0,...,0,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,11362.473684,10737.789474,2.0,1,3
1,M5B,The Senator Restaurant,Diner,249 Victoria Street,btwn Dundas St E and Shuter St,170.0,43.655641,-79.379199,4ad7929cf964a520500c21e3,94.0,...,0,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,11362.473684,10737.789474,2.0,1,3
2,M5K,Epic,Wine Bar,100 Front Street West,NaN,187.0,43.645515,-79.381239,4ad4c05df964a5203af620e3,28.0,...,1,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,13028.000000,24851.250000,4.0,2,1
3,M5K,McDonald's,Fast Food Restaurant,61 Front St. W.,in Union Station,182.0,43.646146,-79.379822,4b8d36f8f964a520c7ed32e3,8.0,...,0,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,13028.000000,24851.250000,4.0,2,1
4,M5K,Stratus,Restaurant,79 Wellington St W,at Piper St,126.0,43.646181,-79.380809,4b231e82f964a5208e5324e3,4.0,...,0,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,13028.000000,24851.250000,4.0,2,1


In [92]:
#these might change depending on the categories of the foursquare search
hotfullrest=fullrest[['Postalcode', 'name', 'categories', 'address',
       'distance', 'lat', 'lng', 'id', 'numbertips', 'rating', 'profitrate',
       'AmericanRestaurant','Breakfast Spot','OtherRestaurant','AsianRestaurant','Bar/Club', 'Borough',
       'Neighborhood', 'VehVol', 'PedVol',
       'NumberRestaurants', 'labels', 'labels2']].copy()

### One of the issues is for each restaurant we are using the mean traffic in the borough, lets fix that

In [93]:
tidyrest.head()

,Postalcode,name,categories,address,crossStreet,distance,lat,lng,id,numbertips,rating,profitrate,AmericanRestaurant,Breakfast Spot,OtherRestaurant,AsianRestaurant,Bar/Club
0,M5B,Subway,Sandwich Place,10 Dundas St. E,at Yonge St.,188.0,43.656260,-79.380920,4c6abb8c2c29d13ad2720e41,3.0,5.8,17.4,1,0,0,0,0
1,M5K,Epic,Wine Bar,100 Front Street West,NaN,187.0,43.645515,-79.381239,4ad4c05df964a5203af620e3,28.0,6.9,193.2,0,0,0,0,1
2,M5K,McDonald's,Fast Food Restaurant,61 Front St. W.,in Union Station,182.0,43.646146,-79.379822,4b8d36f8f964a520c7ed32e3,8.0,4.7,37.6,1,0,0,0,0
3,M5L,Ninki Sushi,Japanese Restaurant,40 King St W,at Bay St,181.0,43.649812,-79.379518,4beb2c0ba9900f475b7a1740,15.0,5.7,85.5,0,0,0,1,0
4,M5L,Over Easy,Breakfast Spot,56 Yonge St,at Melinda St,179.0,43.648077,-79.377599,4b06e099f964a52002f222e3,32.0,7.7,246.4,0,1,0,0,0


In [94]:
newrest=tidyrest.copy()
trafficdf=trafficdataframe.copy()
trafficdf.columns=['TCS #', 'Main', 'Midblock Route', 'Side 1 Route', 'Side 2 Route',
       'Activation Date', 'Latitude', 'Longitude', 'Count Date',
       'vehvol', 'pedvol', 'ClosestPC']
trafficdf.columns

Index(['TCS #', 'Main', 'Midblock Route', 'Side 1 Route', 'Side 2 Route',
       'Activation Date', 'Latitude', 'Longitude', 'Count Date', 'vehvol',
       'pedvol', 'ClosestPC'],
      dtype='object')

### Now we assign each restaurant the traffic values of the closest street in the traffic data frame

In [95]:
newrest['PedVol']=0
newrest['VehVol']=0
for  row1 in newrest.iterrows():
    #We initialize the starting value to a high enough value that all dist will be below it
    shortdist=100
    for  row2 in trafficdf.iterrows():
        dist=(row1[1].lat-row2[1].Latitude)**2+(row1[1].lng-row2[1].Longitude)**2
        if dist<shortdist:
            shortdist=dist
            pedvol=row2[1].pedvol
            vehvol=row2[1].vehvol
    newrest.at[row1[0],'PedVol']=pedvol
    newrest.at[row1[0],'VehVol']=vehvol



In [96]:
newrest.head()

,Postalcode,name,categories,address,crossStreet,distance,lat,lng,id,numbertips,rating,profitrate,AmericanRestaurant,Breakfast Spot,OtherRestaurant,AsianRestaurant,Bar/Club,PedVol,VehVol
0,M5B,Subway,Sandwich Place,10 Dundas St. E,at Yonge St.,188.0,43.656260,-79.380920,4c6abb8c2c29d13ad2720e41,3.0,5.8,17.4,1,0,0,0,0,34615,10925
1,M5K,Epic,Wine Bar,100 Front Street West,NaN,187.0,43.645515,-79.381239,4ad4c05df964a5203af620e3,28.0,6.9,193.2,0,0,0,0,1,17564,16857
2,M5K,McDonald's,Fast Food Restaurant,61 Front St. W.,in Union Station,182.0,43.646146,-79.379822,4b8d36f8f964a520c7ed32e3,8.0,4.7,37.6,1,0,0,0,0,17115,18965
3,M5L,Ninki Sushi,Japanese Restaurant,40 King St W,at Bay St,181.0,43.649812,-79.379518,4beb2c0ba9900f475b7a1740,15.0,5.7,85.5,0,0,0,1,0,11224,12454
4,M5L,Over Easy,Breakfast Spot,56 Yonge St,at Melinda St,179.0,43.648077,-79.377599,4b06e099f964a52002f222e3,32.0,7.7,246.4,0,1,0,0,0,26656,13620


### We assign a categoric viability column for Machine learning algorithms, such as decission trees

In [97]:
catrest=newrest.copy()
catrest['categoricprofit']='Empty'
for ind,row in newrest.iterrows():
    if row.profitrate<200:
        catrest.at[ind,'categoricprofit']='Failure'
    elif row.profitrate<500:
        catrest.at[ind,'categoricprofit']='Viable'
    else:
        catrest.at[ind,'categoricprofit']='Success'

catrest.head()

### We save all the dataframes we will use in the study of our problem

In [98]:
newrest.to_csv('newrest.csv')

In [99]:
restlist.to_csv('restlist.csv')

In [107]:
onlytoronto.to_csv('onlytoronto.csv')

In [101]:
catrest.to_csv('catrest.csv')

In [102]:
fullrest.to_csv('fullrest.csv')

In [103]:
hotfullrest.to_csv('hotfullrest.csv')

In [104]:
tidyrest.to_csv('tidyrest.csv')

In [105]:
torontotraff.to_csv('torontotraff.csv')

In [106]:
restdentoronto.to_csv('restdentoronto.csv')